In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def dydx(x,y):
    #set the derivatives to integrate
    #equation is d^2y/dx^2 = -y
    #so we can write
    #dydx = z
    #dzdx = -y
    #we will set y = y[0]
    #we will set z = y[1]
    
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #return an array
    return y_derivs

In [ ]:
def rk6_mv_core(dydx,xi,yi,nv,h,cd,star):
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh2 = xi + cd['c'][1]*h
    x_ipoh3 = xi + cd['c'][2]*h
    x_ipoh4 = xi + cd['c'][3]*h
    x_ipoh5 = xi + cd['c'][4]*h
    x_ipoh6 = xi + cd['c'][5]*h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + cd['a'][1][0]*k1[:]
    y_derivs = dydx(x_ipoh2,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + cd['a'][2][0]*k1[:] + cd['a'][2][1]*k2[:]
    y_derivs = dydx(x_ipoh3,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + cd['a'][3][0]*k1[:] + cd['a'][3][1]*k2[:] + cd['a'][3][2]*k3[:]
    y_derivs = dydx(x_ipoh4,y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    y_temp[:] = yi[:] + cd['a'][4][0]*k1[:] + cd['a'][4][1]*k2[:] + cd['a'][4][2]*k3[:] + cd['a'][4][3]*k4[:]
    y_derivs = dydx(x_ipoh5,y_temp)
    k5[:] = h*y_derivs[:]
    
    #get k6 values
    y_temp[:] = yi[:] + cd['a'][5][0]*k1[:] + cd['a'][5][1]*k2[:] + cd['a'][5][2]*k3[:] + cd['a'][5][3]*k4[:] + cd['a'][5][4]*k5[:]
    y_derivs = dydx(x_ipoh6,y_temp)
    k6[:] = h*y_derivs[:]
    
    #advance y by a step h
    if(star==False):
        yipo = yi + cd['b'][0]*k1 + cd['b'][1]*k2 + cd['b'][2]*k3 + cd['b'][3]*k4 + cd['b'][4]*k5 + cd['b'][5]*k6
    else:
        yipo = yi + cd['b_star'][0]*k1 + cd['b_star'][1]*k2 + cd['b_star'][2]*k3 + cd['b_star'][3]*k4 + cd['b_star'][4]*k5 + cd['b_star'][5]*k6
    
    return yipo

In [ ]:
def rk6_mv_ad(dydx,x_i,y_i,nv,h,tol,cd):
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        #estimate our error by taking one step of size h vs. two steps of size h/2
        y_2 = rk6_mv_core(dydx,x_i,y_i,nv,h_step,cd,False)
        #y_1 = rk6_mv_core(dydx,x_i,y_i,nv,0.5*h_step,cd)
        #y_11 = rk6_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step,cd)
        y_star = rk6_mv_core(dydx,x_i,y_i,nv,h_step,cd,True)
        
        #compute an error
        Delta = np.fabs(y_2 - y_star)
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
        
        #check iteration
        if(i>=imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
        
        #iterate
        i+=1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

In [ ]:
def rk6_mv(dydx,a,b,y_a,tol,cd):
    #dydx is the derivative wrt x
    #a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y

    #define starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small!
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled ODEs to the size of y_a
    nv = len(y_a)
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = True
    
    #loop until we reach the right side
    while(flag):
        #calculate y_i+1:
        yi_new, h_new, h_step = rk6_mv_ad(dydx,xi,yi,nv,h,tol,cd)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            #take a smaller step
            h = b-xi
            #recalculate y_i+1
            yi_new, h_new, h_step = rk6_mv_ad(dydx,xi,yi,nv,h,tol,cd)
            
            #break
            flag = False
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many iterations
        if(i>=imax):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
        
        #iterate
        i+=1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f"%(i,xi,h_step,b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = False
        
    #return the answer
    return x,y

In [ ]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#dictionary of arrays of coefficients
cd =   {'c' : [0.,.2,.3,.6,1,.875],
        'a' : [[0,0,0,0,0],[.2,0,0,0,0],[3./40.,9./40.,0,0,0],[.3,-.9,1.2,0,0],[-11./54.,2.5,-70./27.,35./27.,0],[7./8.,1631./55296.,175./512.,575./13824.,44275./110592.,253./4096.]],
        'b' : [37./378.,0.,250./621.,125./594.,0.,512./1771.],
        'b_star' : [2825./27648.,0.,18575./48384.,13525./55296.,277./14336.,.25]}

#perform the integration
x,y = rk6_mv(dydx,a,b,y_0,tolerance,cd)

In [ ]:
#plot the result
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

In [ ]:
#plot the error
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x,y_error,label="y(x) Error")
plt.plot(x,dydx_error,label="dydx(x) Error")
plt.legend(frameon=False)